# Outlier Analysis

Questions:
* If you found NULL values after you joined the datasets, what did you do with them? How many were there?

After joining the datasets, there were 750332 null values in the SA2 dataset and 182 in the origninal dataset, these comprised of approximatley 2% of the total dataset. for null values in the SA2 dataset, the postcodes were used to determine which state the purchase came from. then averaging over all the data in that state the other columns were populated.

* Was there any missing data that shouldn’t be missing after joining to your external
dataset? If anything was missing, how much was there and what did you do about it?

there was only 182 rows of missing data once joined, due to the extremely low number and the fact that the whole purchase was null, they were removed.

* If you decided to omit outliers, what does the distribution look like prior and after?

due to only removed a few records from the data, the distribution does not change.



In [2]:
from pyspark.sql import SparkSession, functions as F, DataFrame
from pyspark.sql.functions import col

spark = (
    SparkSession.builder.appName("Customer_Analysis")
    .config("spark.sql.repl.eagerEval.enabled", True) 
    .config("spark.sql.parquet.cacheMetadata", "true")
    .config("spark.sql.session.timeZone", "Etc/UTC")
    .config("spark.executor.memory", "2g")
    .config("spark.driver.memory", "4g")
    .getOrCreate()
)

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


22/09/12 13:07:52 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [67]:
data = spark.read.parquet('../data/curated/final_data.parquet')

In [68]:
data

SA2_CODE_2021,postcode,consumer_id,user_id,merchant_abn,dollar_value,order_id,order_datetime,name,tags,consumer_name,address,gender,SA2_NAME_2021,Median_tot_prsnl_inc_weekly,Median_rent_weekly,Median_mortgage_repay_monthly,Median_age_persons,Median_tot_hhd_inc_weekly,Average_household_size,Completed Year 12,Did Not Attend School,TOT_P_P,Age_0_4_yr_P,Age_5_14_yr_P,Age_15_19_yr_P,Age_20_24_yr_P,Age_25_34_yr_P,Age_35_44_yr_P,Age_45_54_yr_P,Age_55_64_yr_P,Age_65_74_yr_P,Age_75_84_yr_P,Age_85ov_P
null,852,1140,6421,23338656015,240.79240453805204,c22bbaa1-8e70-49b...,2021-08-19,Iaculis LLC,"[watch, clock, ...",Christopher Powell,880 Brandon Stree...,Undisclosed,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null
null,7321,94215,11809,49891706470,48.86426156234827,5cfef23a-a07c-455...,2021-05-21,Non Vestibulum In...,[tent and awning ...,Amy Nguyen,85806 Melton Squares,Female,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null
null,6872,799687,24034,38090089066,118.19708950112054,ece422e2-c9e8-42c...,2021-04-02,Interdum Feugiat ...,"[furniture, home...",Angela Little,337 Carter Walk,Female,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null
null,7330,334995,4041,33357097824,24.599565257317405,7fd18add-d03c-479...,2021-07-29,Sem Ut Incorporated,"[hobby, toy and ...",Eric Cooper,137 Brian Turnpike,Male,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null
null,852,1140,6421,48534649627,269.8021779483706,dcc21448-04c5-42d...,2021-08-14,Dignissim Maecena...,"[opticians, opti...",Christopher Powell,880 Brandon Stree...,Undisclosed,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null
null,7321,94215,11809,49891706470,48.86426156234827,5cfef23a-a07c-455...,2021-05-21,Non Vestibulum In...,[tent and awning ...,Amy Nguyen,85806 Melton Squares,Female,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null
null,6872,799687,24034,89726005175,58.42108971934746,912a54f6-e059-47f...,2021-03-27,Est Nunc Consulting,[tent and awning ...,Angela Little,337 Carter Walk,Female,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null
null,7330,334995,4041,33357097824,24.599565257317405,7fd18add-d03c-479...,2021-07-29,Sem Ut Incorporated,"[hobby, toy and ...",Eric Cooper,137 Brian Turnpike,Male,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null
null,852,1140,6421,82065156333,13.141362673704986,343f78b0-8b73-44f...,2021-08-15,Nascetur Ridiculu...,[tent and awning ...,Christopher Powell,880 Brandon Stree...,Undisclosed,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null
null,7321,94215,11809,49891706470,48.86426156234827,5cfef23a-a07c-455...,2021-05-21,Non Vestibulum In...,[tent and awning ...,Amy Nguyen,85806 Melton Squares,Female,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null


In [69]:
data.na.drop(subset=["SA2_CODE_2021"])

SA2_CODE_2021,postcode,consumer_id,user_id,merchant_abn,dollar_value,order_id,order_datetime,name,tags,consumer_name,address,gender,SA2_NAME_2021,Median_tot_prsnl_inc_weekly,Median_rent_weekly,Median_mortgage_repay_monthly,Median_age_persons,Median_tot_hhd_inc_weekly,Average_household_size,Completed Year 12,Did Not Attend School,TOT_P_P,Age_0_4_yr_P,Age_5_14_yr_P,Age_15_19_yr_P,Age_20_24_yr_P,Age_25_34_yr_P,Age_35_44_yr_P,Age_45_54_yr_P,Age_55_64_yr_P,Age_65_74_yr_P,Age_75_84_yr_P,Age_85ov_P
102011033,2250,957666,23534,29641342749,228.13576584758118,8283052a-431c-4f9...,2021-08-20,Et Associates,[computer program...,Paul Jefferson,057 Patel Common ...,Male,Kariong,850,450,2080,36,2200,3,2736,12,6485,414,1009,519,403,756,936,968,804,418,195,74
102011033,2250,957666,23534,29641342749,228.13576584758118,8283052a-431c-4f9...,2021-08-20,Et Associates,[computer program...,Paul Jefferson,057 Patel Common ...,Male,Kariong,850,450,2080,36,2200,3,2736,12,6485,414,1009,519,403,756,936,968,804,418,195,74
102011033,2250,957666,23534,66034737896,43.57631530803281,ccd8b43e-88fa-4a9...,2021-08-19,Luctus Ut LLC,"[watch, clock, ...",Paul Jefferson,057 Patel Common ...,Male,Kariong,850,450,2080,36,2200,3,2736,12,6485,414,1009,519,403,756,936,968,804,418,195,74
102011033,2250,957666,23534,66034737896,43.57631530803281,ccd8b43e-88fa-4a9...,2021-08-19,Luctus Ut LLC,"[watch, clock, ...",Paul Jefferson,057 Patel Common ...,Male,Kariong,850,450,2080,36,2200,3,2736,12,6485,414,1009,519,403,756,936,968,804,418,195,74
102011033,2250,957666,23534,56945597985,14.980127758236039,adf95737-385f-407...,2021-08-22,Sem LLP,"[cable, satellit...",Paul Jefferson,057 Patel Common ...,Male,Kariong,850,450,2080,36,2200,3,2736,12,6485,414,1009,519,403,756,936,968,804,418,195,74
102011033,2250,957666,23534,56945597985,14.980127758236039,adf95737-385f-407...,2021-08-22,Sem LLP,"[cable, satellit...",Paul Jefferson,057 Patel Common ...,Male,Kariong,850,450,2080,36,2200,3,2736,12,6485,414,1009,519,403,756,936,968,804,418,195,74
102011033,2250,957666,23534,60978195146,3.2940935801059776,ecfada9b-44ce-42d...,2021-08-15,Lobortis Tellus C...,"[shoe shops, c, ...",Paul Jefferson,057 Patel Common ...,Male,Kariong,850,450,2080,36,2200,3,2736,12,6485,414,1009,519,403,756,936,968,804,418,195,74
102011033,2250,957666,23534,60978195146,3.2940935801059776,ecfada9b-44ce-42d...,2021-08-15,Lobortis Tellus C...,"[shoe shops, c, ...",Paul Jefferson,057 Patel Common ...,Male,Kariong,850,450,2080,36,2200,3,2736,12,6485,414,1009,519,403,756,936,968,804,418,195,74
102011033,2250,957666,23534,68435002949,433.1880172503183,ef2e7b0d-9d44-440...,2021-08-27,Aliquam Eu Inc.,[artist supply an...,Paul Jefferson,057 Patel Common ...,Male,Kariong,850,450,2080,36,2200,3,2736,12,6485,414,1009,519,403,756,936,968,804,418,195,74
102011033,2250,957666,23534,68435002949,433.1880172503183,ef2e7b0d-9d44-440...,2021-08-27,Aliquam Eu Inc.,[artist supply an...,Paul Jefferson,057 Patel Common ...,Male,Kariong,850,450,2080,36,2200,3,2736,12,6485,414,1009,519,403,756,936,968,804,418,195,74


In [70]:
from pyspark.sql.functions import isnan, when, count, col
data.select([count(when(col(c).isNull(), c)).alias(c) for c in data.columns])

SA2_CODE_2021,postcode,consumer_id,user_id,merchant_abn,dollar_value,order_id,order_datetime,name,tags,consumer_name,address,gender,SA2_NAME_2021,Median_tot_prsnl_inc_weekly,Median_rent_weekly,Median_mortgage_repay_monthly,Median_age_persons,Median_tot_hhd_inc_weekly,Average_household_size,Completed Year 12,Did Not Attend School,TOT_P_P,Age_0_4_yr_P,Age_5_14_yr_P,Age_15_19_yr_P,Age_20_24_yr_P,Age_25_34_yr_P,Age_35_44_yr_P,Age_45_54_yr_P,Age_55_64_yr_P,Age_65_74_yr_P,Age_75_84_yr_P,Age_85ov_P
750332,182,182,182,182,182,182,182,182,182,182,182,182,750332,750333,750333,750333,750333,750333,750333,750333,750333,750333,750333,750333,750333,750333,750333,750333,750333,750333,750333,750333,750333


## SA2_CODE_2021 / SA2_NAME_2021

In [71]:
from pyspark.sql.functions import col
data.filter(col("SA2_CODE_2021").isNull()).count()/data.count()

0.019819309386088536

In [72]:
import pyspark.sql.functions as f
import pandas as pd
data = data.withColumn(
    'state',
    F.when(((col('postcode') >= 1000) & (col('postcode') <= 1999)) | ((col('postcode') >= 2000) & (col('postcode') <= 2599)) | ((col('postcode') >= 2619) & (col('postcode') < 2899)) | ((col('postcode') >= 2921) & (col('postcode') < 2999)), 'NSW')\
    .when(((col('postcode') >= 200) & (col('postcode') <= 299)) | ((col('postcode') >= 2600) & (col('postcode') <= 2618)) | ((col('postcode') >= 2900) & (col('postcode') < 2920)), 'ACT')\
    .when(((col('postcode') >= 3000) & (col('postcode') <= 3999)) | ((col('postcode') >= 8000) & (col('postcode') <= 8999)), 'VIC')\
    .when(((col('postcode') >= 4000) & (col('postcode') <= 4999)) | ((col('postcode') >= 9000) & (col('postcode') <= 9999)), 'QLD')\
    .when(((col('postcode') >= 5000) & (col('postcode') <= 5999)), 'SA')\
    .when(((col('postcode') >= 6000) & (col('postcode') <= 6999)), 'WA')\
    .when(((col('postcode') >= 7000) & (col('postcode') <= 7999)), 'TAS')\
    .otherwise('NT')
)
data = data.withColumn(
    'SA2_NAME_2021',
    F.when( (col('state') == 'VIC') & (col('SA2_NAME_2021').isNull()), 'Victoria')\
    .when( (col('state') == 'NSW') & (col('SA2_NAME_2021').isNull()), 'New South Wales')\
    .when( (col('state') == 'QLD') & (col('SA2_NAME_2021').isNull()), 'Queensland')\
    .when( (col('state') == 'NT') & (col('SA2_NAME_2021').isNull()), 'Northern Territory')\
    .when( (col('state') == 'WA') & (col('SA2_NAME_2021').isNull()), 'Western Austraia')\
    .when( (col('state') == 'SA') & (col('SA2_NAME_2021').isNull()), 'South Australia')\
    .when( (col('state') == 'TAS') & (col('SA2_NAME_2021').isNull()), 'Tasmania')\
    .when( (col('state') == 'ACT') & (col('SA2_NAME_2021').isNull()), 'Australian Capital Territory')\
    .otherwise(col('SA2_NAME_2021'))
)
data.limit(5)

SA2_CODE_2021,postcode,consumer_id,user_id,merchant_abn,dollar_value,order_id,order_datetime,name,tags,consumer_name,address,gender,SA2_NAME_2021,Median_tot_prsnl_inc_weekly,Median_rent_weekly,Median_mortgage_repay_monthly,Median_age_persons,Median_tot_hhd_inc_weekly,Average_household_size,Completed Year 12,Did Not Attend School,TOT_P_P,Age_0_4_yr_P,Age_5_14_yr_P,Age_15_19_yr_P,Age_20_24_yr_P,Age_25_34_yr_P,Age_35_44_yr_P,Age_45_54_yr_P,Age_55_64_yr_P,Age_65_74_yr_P,Age_75_84_yr_P,Age_85ov_P,state
null,852,1140,6421,23338656015,240.79240453805204,c22bbaa1-8e70-49b...,2021-08-19,Iaculis LLC,"[watch, clock, ...",Christopher Powell,880 Brandon Stree...,Undisclosed,Northern Territory,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,NT
null,7321,94215,11809,49891706470,48.86426156234827,5cfef23a-a07c-455...,2021-05-21,Non Vestibulum In...,[tent and awning ...,Amy Nguyen,85806 Melton Squares,Female,Tasmania,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,TAS
null,6872,799687,24034,38090089066,118.19708950112054,ece422e2-c9e8-42c...,2021-04-02,Interdum Feugiat ...,"[furniture, home...",Angela Little,337 Carter Walk,Female,Western Austraia,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,WA
null,7330,334995,4041,33357097824,24.599565257317405,7fd18add-d03c-479...,2021-07-29,Sem Ut Incorporated,"[hobby, toy and ...",Eric Cooper,137 Brian Turnpike,Male,Tasmania,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,TAS
null,852,1140,6421,48534649627,269.8021779483706,dcc21448-04c5-42d...,2021-08-14,Dignissim Maecena...,"[opticians, opti...",Christopher Powell,880 Brandon Stree...,Undisclosed,Northern Territory,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,NT


In [73]:
data.na.drop(subset=["SA2_CODE_2021"])

SA2_CODE_2021,postcode,consumer_id,user_id,merchant_abn,dollar_value,order_id,order_datetime,name,tags,consumer_name,address,gender,SA2_NAME_2021,Median_tot_prsnl_inc_weekly,Median_rent_weekly,Median_mortgage_repay_monthly,Median_age_persons,Median_tot_hhd_inc_weekly,Average_household_size,Completed Year 12,Did Not Attend School,TOT_P_P,Age_0_4_yr_P,Age_5_14_yr_P,Age_15_19_yr_P,Age_20_24_yr_P,Age_25_34_yr_P,Age_35_44_yr_P,Age_45_54_yr_P,Age_55_64_yr_P,Age_65_74_yr_P,Age_75_84_yr_P,Age_85ov_P,state
102011033,2250,957666,23534,29641342749,228.13576584758118,8283052a-431c-4f9...,2021-08-20,Et Associates,[computer program...,Paul Jefferson,057 Patel Common ...,Male,Kariong,850,450,2080,36,2200,3,2736,12,6485,414,1009,519,403,756,936,968,804,418,195,74,NSW
102011033,2250,957666,23534,29641342749,228.13576584758118,8283052a-431c-4f9...,2021-08-20,Et Associates,[computer program...,Paul Jefferson,057 Patel Common ...,Male,Kariong,850,450,2080,36,2200,3,2736,12,6485,414,1009,519,403,756,936,968,804,418,195,74,NSW
102011033,2250,957666,23534,66034737896,43.57631530803281,ccd8b43e-88fa-4a9...,2021-08-19,Luctus Ut LLC,"[watch, clock, ...",Paul Jefferson,057 Patel Common ...,Male,Kariong,850,450,2080,36,2200,3,2736,12,6485,414,1009,519,403,756,936,968,804,418,195,74,NSW
102011033,2250,957666,23534,66034737896,43.57631530803281,ccd8b43e-88fa-4a9...,2021-08-19,Luctus Ut LLC,"[watch, clock, ...",Paul Jefferson,057 Patel Common ...,Male,Kariong,850,450,2080,36,2200,3,2736,12,6485,414,1009,519,403,756,936,968,804,418,195,74,NSW
102011033,2250,957666,23534,56945597985,14.980127758236039,adf95737-385f-407...,2021-08-22,Sem LLP,"[cable, satellit...",Paul Jefferson,057 Patel Common ...,Male,Kariong,850,450,2080,36,2200,3,2736,12,6485,414,1009,519,403,756,936,968,804,418,195,74,NSW
102011033,2250,957666,23534,56945597985,14.980127758236039,adf95737-385f-407...,2021-08-22,Sem LLP,"[cable, satellit...",Paul Jefferson,057 Patel Common ...,Male,Kariong,850,450,2080,36,2200,3,2736,12,6485,414,1009,519,403,756,936,968,804,418,195,74,NSW
102011033,2250,957666,23534,60978195146,3.2940935801059776,ecfada9b-44ce-42d...,2021-08-15,Lobortis Tellus C...,"[shoe shops, c, ...",Paul Jefferson,057 Patel Common ...,Male,Kariong,850,450,2080,36,2200,3,2736,12,6485,414,1009,519,403,756,936,968,804,418,195,74,NSW
102011033,2250,957666,23534,60978195146,3.2940935801059776,ecfada9b-44ce-42d...,2021-08-15,Lobortis Tellus C...,"[shoe shops, c, ...",Paul Jefferson,057 Patel Common ...,Male,Kariong,850,450,2080,36,2200,3,2736,12,6485,414,1009,519,403,756,936,968,804,418,195,74,NSW
102011033,2250,957666,23534,68435002949,433.1880172503183,ef2e7b0d-9d44-440...,2021-08-27,Aliquam Eu Inc.,[artist supply an...,Paul Jefferson,057 Patel Common ...,Male,Kariong,850,450,2080,36,2200,3,2736,12,6485,414,1009,519,403,756,936,968,804,418,195,74,NSW
102011033,2250,957666,23534,68435002949,433.1880172503183,ef2e7b0d-9d44-440...,2021-08-27,Aliquam Eu Inc.,[artist supply an...,Paul Jefferson,057 Patel Common ...,Male,Kariong,850,450,2080,36,2200,3,2736,12,6485,414,1009,519,403,756,936,968,804,418,195,74,NSW


In [74]:
data = data.withColumn("Median_tot_prsnl_inc_weekly", data.Median_tot_prsnl_inc_weekly.cast('float'))
data = data.withColumn("Median_rent_weekly", data.Median_rent_weekly.cast('float'))
data = data.withColumn("Median_mortgage_repay_monthly",data.Median_mortgage_repay_monthly.cast('float'))
data = data.withColumn("Median_age_persons",data.Median_age_persons.cast('float'))
data = data.withColumn("Median_tot_hhd_inc_weekly",data.Median_tot_hhd_inc_weekly.cast('float'))
data = data.withColumn("Average_household_size",data.Average_household_size.cast('float'))
data = data.withColumnRenamed("Completed Year 12", "Completed_Year_12")
data = data.withColumn("Completed_Year_12",data.Completed_Year_12.cast('float'))
data = data.withColumnRenamed("Did Not Attend School", "Did_Not_Attend_School")
data = data.withColumn("Did_Not_Attend_School",data.Did_Not_Attend_School.cast('float'))
data = data.withColumn("TOT_P_P",data.TOT_P_P.cast('float'))


In [75]:
columns = ["Median_tot_prsnl_inc_weekly", "Median_rent_weekly", "Median_mortgage_repay_monthly", "Median_age_persons", "Median_tot_hhd_inc_weekly",\
           "Average_household_size", "Completed_Year_12", "Did_Not_Attend_School", "TOT_P_P"]

for column in columns:
    data = data.withColumn(
        column,
        F.when( (col('SA2_NAME_2021') == 'Northern Territory'), data.groupBy('state').mean(column).collect()[0][1])\
        .when( (col('SA2_NAME_2021') == 'Australian Capital Territory'), data.groupBy('state').mean(column).collect()[1][1])\
        .when( (col('SA2_NAME_2021') == 'South Australia'), data.groupBy('state').mean(column).collect()[2][1])\
        .when( (col('SA2_NAME_2021') == 'Tasmania'), data.groupBy('state').mean(column).collect()[3][1])\
        .when( (col('SA2_NAME_2021') == 'Western Austraia'), data.groupBy('state').mean(column).collect()[4][1])\
        .when( (col('SA2_NAME_2021') == 'Queensland'), data.groupBy('state').mean(column).collect()[5][1])\
        .when( (col('SA2_NAME_2021') == 'Victoria'), data.groupBy('state').mean(column).collect()[6][1])\
        .when( (col('SA2_NAME_2021') == 'New South Wales'), data.groupBy('state').mean(column).collect()[7][1])\
        .otherwise(col(column))
    )
data

SA2_CODE_2021,postcode,consumer_id,user_id,merchant_abn,dollar_value,order_id,order_datetime,name,tags,consumer_name,address,gender,SA2_NAME_2021,Median_tot_prsnl_inc_weekly,Median_rent_weekly,Median_mortgage_repay_monthly,Median_age_persons,Median_tot_hhd_inc_weekly,Average_household_size,Completed_Year_12,Did_Not_Attend_School,TOT_P_P,Age_0_4_yr_P,Age_5_14_yr_P,Age_15_19_yr_P,Age_20_24_yr_P,Age_25_34_yr_P,Age_35_44_yr_P,Age_45_54_yr_P,Age_55_64_yr_P,Age_65_74_yr_P,Age_75_84_yr_P,Age_85ov_P,state
null,852,1140,6421,23338656015,240.79240453805204,c22bbaa1-8e70-49b...,2021-08-19,Iaculis LLC,"[watch, clock, ...",Christopher Powell,880 Brandon Stree...,Undisclosed,Northern Territory,645.9527090071066,87.9500782220102,478.8819373802993,31.990788744449045,1277.1489375135925,3.4840981139623093,709.3371263408235,37.410346351627226,3169.560259009562,null,null,null,null,null,null,null,null,null,null,null,NT
null,7321,94215,11809,49891706470,48.86426156234827,5cfef23a-a07c-455...,2021-05-21,Non Vestibulum In...,[tent and awning ...,Amy Nguyen,85806 Melton Squares,Female,Tasmania,654.7627923426741,247.91508634682842,1182.5427094138233,46.957218003637045,1225.7466943272127,2.277830912050027,1654.4454957915325,10.896212247181818,5081.844906685248,null,null,null,null,null,null,null,null,null,null,null,TAS
null,6872,799687,24034,38090089066,118.19708950112054,ece422e2-c9e8-42c...,2021-04-02,Interdum Feugiat ...,"[furniture, home...",Angela Little,337 Carter Walk,Female,Western Austraia,836.7966870108314,235.5527510544241,1437.028984063036,42.581815939994776,1530.0211433587413,2.4394982812246875,2478.71725636885,28.07392873978152,6548.73302066646,null,null,null,null,null,null,null,null,null,null,null,WA
null,7330,334995,4041,33357097824,24.599565257317405,7fd18add-d03c-479...,2021-07-29,Sem Ut Incorporated,"[hobby, toy and ...",Eric Cooper,137 Brian Turnpike,Male,Tasmania,654.7627923426741,247.91508634682842,1182.5427094138233,46.957218003637045,1225.7466943272127,2.277830912050027,1654.4454957915325,10.896212247181818,5081.844906685248,null,null,null,null,null,null,null,null,null,null,null,TAS
null,852,1140,6421,48534649627,269.8021779483706,dcc21448-04c5-42d...,2021-08-14,Dignissim Maecena...,"[opticians, opti...",Christopher Powell,880 Brandon Stree...,Undisclosed,Northern Territory,645.9527090071066,87.9500782220102,478.8819373802993,31.990788744449045,1277.1489375135925,3.4840981139623093,709.3371263408235,37.410346351627226,3169.560259009562,null,null,null,null,null,null,null,null,null,null,null,NT
null,7321,94215,11809,49891706470,48.86426156234827,5cfef23a-a07c-455...,2021-05-21,Non Vestibulum In...,[tent and awning ...,Amy Nguyen,85806 Melton Squares,Female,Tasmania,654.7627923426741,247.91508634682842,1182.5427094138233,46.957218003637045,1225.7466943272127,2.277830912050027,1654.4454957915325,10.896212247181818,5081.844906685248,null,null,null,null,null,null,null,null,null,null,null,TAS
null,6872,799687,24034,89726005175,58.42108971934746,912a54f6-e059-47f...,2021-03-27,Est Nunc Consulting,[tent and awning ...,Angela Little,337 Carter Walk,Female,Western Austraia,836.7966870108314,235.5527510544241,1437.028984063036,42.581815939994776,1530.0211433587413,2.4394982812246875,2478.71725636885,28.07392873978152,6548.73302066646,null,null,null,null,null,null,null,null,null,null,null,WA
null,7330,334995,4041,33357097824,24.599565257317405,7fd18add-d03c-479...,2021-07-29,Sem Ut Incorporated,"[hobby, toy and ...",Eric Cooper,137 Brian Turnpike,Male,Tasmania,654.7627923426741,247.91508634682842,1182.5427094138233,46.957218003637045,1225.7466943272127,2.277830912050027,1654.4454957915325,10.896212247181818,5081.844906685248,null,null,null,null,null,null,null,null,null,null,null,TAS
null,852,1140,6421,82065156333,13.141362673704986,343f78b0-8b73-44f...,2021-08-15,Nascetur Ridiculu...,[tent and awning ...,Christopher Powell,880 Brandon Stree...,Undisclosed,Northern Territory,645.9527090071066,87.9500782220102,478.8819373802993,31.99078874444904

In [76]:
data.na.drop(subset=["SA2_CODE_2021"])

SA2_CODE_2021,postcode,consumer_id,user_id,merchant_abn,dollar_value,order_id,order_datetime,name,tags,consumer_name,address,gender,SA2_NAME_2021,Median_tot_prsnl_inc_weekly,Median_rent_weekly,Median_mortgage_repay_monthly,Median_age_persons,Median_tot_hhd_inc_weekly,Average_household_size,Completed_Year_12,Did_Not_Attend_School,TOT_P_P,Age_0_4_yr_P,Age_5_14_yr_P,Age_15_19_yr_P,Age_20_24_yr_P,Age_25_34_yr_P,Age_35_44_yr_P,Age_45_54_yr_P,Age_55_64_yr_P,Age_65_74_yr_P,Age_75_84_yr_P,Age_85ov_P,state
102011033,2250,957666,23534,29641342749,228.13576584758118,8283052a-431c-4f9...,2021-08-20,Et Associates,[computer program...,Paul Jefferson,057 Patel Common ...,Male,Kariong,850.0,450.0,2080.0,36.0,2200.0,3.0,2736.0,12.0,6485.0,414,1009,519,403,756,936,968,804,418,195,74,NSW
102011033,2250,957666,23534,29641342749,228.13576584758118,8283052a-431c-4f9...,2021-08-20,Et Associates,[computer program...,Paul Jefferson,057 Patel Common ...,Male,Kariong,850.0,450.0,2080.0,36.0,2200.0,3.0,2736.0,12.0,6485.0,414,1009,519,403,756,936,968,804,418,195,74,NSW
102011033,2250,957666,23534,66034737896,43.57631530803281,ccd8b43e-88fa-4a9...,2021-08-19,Luctus Ut LLC,"[watch, clock, ...",Paul Jefferson,057 Patel Common ...,Male,Kariong,850.0,450.0,2080.0,36.0,2200.0,3.0,2736.0,12.0,6485.0,414,1009,519,403,756,936,968,804,418,195,74,NSW
102011033,2250,957666,23534,66034737896,43.57631530803281,ccd8b43e-88fa-4a9...,2021-08-19,Luctus Ut LLC,"[watch, clock, ...",Paul Jefferson,057 Patel Common ...,Male,Kariong,850.0,450.0,2080.0,36.0,2200.0,3.0,2736.0,12.0,6485.0,414,1009,519,403,756,936,968,804,418,195,74,NSW
102011033,2250,957666,23534,56945597985,14.980127758236039,adf95737-385f-407...,2021-08-22,Sem LLP,"[cable, satellit...",Paul Jefferson,057 Patel Common ...,Male,Kariong,850.0,450.0,2080.0,36.0,2200.0,3.0,2736.0,12.0,6485.0,414,1009,519,403,756,936,968,804,418,195,74,NSW
102011033,2250,957666,23534,56945597985,14.980127758236039,adf95737-385f-407...,2021-08-22,Sem LLP,"[cable, satellit...",Paul Jefferson,057 Patel Common ...,Male,Kariong,850.0,450.0,2080.0,36.0,2200.0,3.0,2736.0,12.0,6485.0,414,1009,519,403,756,936,968,804,418,195,74,NSW
102011033,2250,957666,23534,60978195146,3.2940935801059776,ecfada9b-44ce-42d...,2021-08-15,Lobortis Tellus C...,"[shoe shops, c, ...",Paul Jefferson,057 Patel Common ...,Male,Kariong,850.0,450.0,2080.0,36.0,2200.0,3.0,2736.0,12.0,6485.0,414,1009,519,403,756,936,968,804,418,195,74,NSW
102011033,2250,957666,23534,60978195146,3.2940935801059776,ecfada9b-44ce-42d...,2021-08-15,Lobortis Tellus C...,"[shoe shops, c, ...",Paul Jefferson,057 Patel Common ...,Male,Kariong,850.0,450.0,2080.0,36.0,2200.0,3.0,2736.0,12.0,6485.0,414,1009,519,403,756,936,968,804,418,195,74,NSW
102011033,2250,957666,23534,68435002949,433.1880172503183,ef2e7b0d-9d44-440...,2021-08-27,Aliquam Eu Inc.,[artist supply an...,Paul Jefferson,057 Patel Common ...,Male,Kariong,850.0,450.0,2080.0,36.0,2200.0,3.0,2736.0,12.0,6485.0,414,1009,519,403,756,936,968,804,418,195,74,NSW
102011033,2250,957666,23534,68435002949,433.1880172503183,ef2e7b0d-9d44-440...,2021-08-27,Aliquam Eu Inc.,[artist supply an...,Paul Jefferson,057 Patel Common ...,Male,Kariong,850.0,450.0,2080.0,36.0,2200.0,3.0,2736.0,12.0,6485.0,414,1009,519,403,756,936,968,804,418,195,74,NSW


## Postcode
remove all istances in a null postcode as there is not way to determine the location of the purchase with the postcode.

In [77]:
data = data.na.drop(subset=["postcode"])

In [78]:
print(data.agg({'postcode': 'min'}))
print(data.agg({'postcode': 'max'}))

+-------------+
|min(postcode)|
+-------------+
|         1001|
+-------------+



+-------------+
|max(postcode)|
+-------------+
|         9999|
+-------------+



According to AusPost, all post between 1000-9999 are valid, thus we will keep all.
## ABN
All ABNs are 11 digits long, thus we will verify this:

In [79]:
print(data.agg({'merchant_abn': 'min'}))
print(data.agg({'merchant_abn': 'max'}))

+-----------------+
|min(merchant_abn)|
+-----------------+
|      10023283211|
+-----------------+

+-----------------+
|max(merchant_abn)|
+-----------------+
|      99990536339|
+-----------------+



## Dollar Value

In [80]:
print(data.agg({'dollar_value': 'min'}))
print(data.agg({'dollar_value': 'max'}))

+--------------------+
|   min(dollar_value)|
+--------------------+
|9.756658099412162E-8|
+--------------------+

+------------------+
| max(dollar_value)|
+------------------+
|105193.88578925544|
+------------------+



the above minmum dollar value

In [81]:
data.where(col("dollar_value") <= 0.05).count()

13645

In [82]:
data.where(col("dollar_value") <= 0.05).count()

13645

All these purchases seem unreasonable due the the limited things you can buy for 5c, thus they will be removed from the dataset.

In [83]:
data  = data[data['dollar_value'] >= 0.05]

In [84]:
data.where(col("dollar_value") >= 50000).limit(10)

SA2_CODE_2021,postcode,consumer_id,user_id,merchant_abn,dollar_value,order_id,order_datetime,name,tags,consumer_name,address,gender,SA2_NAME_2021,Median_tot_prsnl_inc_weekly,Median_rent_weekly,Median_mortgage_repay_monthly,Median_age_persons,Median_tot_hhd_inc_weekly,Average_household_size,Completed_Year_12,Did_Not_Attend_School,TOT_P_P,Age_0_4_yr_P,Age_5_14_yr_P,Age_15_19_yr_P,Age_20_24_yr_P,Age_25_34_yr_P,Age_35_44_yr_P,Age_45_54_yr_P,Age_55_64_yr_P,Age_65_74_yr_P,Age_75_84_yr_P,Age_85ov_P,state
null,1212,1319595,23700,11590404675,65660.03308418578,1d8f2322-d8c8-46c...,2021-08-26,Arcu Sed PC,[antique shops - ...,Angela Harris,572 Shannon Plaza,Undisclosed,New South Wales,776.6583292533876,351.68120935335656,1814.3416669907058,44.26223298817287,1625.0275687971643,2.4792831829350526,4383.78123400978,52.53136657119299,10867.791191028951,null,null,null,null,null,null,null,null,null,null,null,NSW
null,6913,1380769,18916,83199298021,59223.92657075801,61fe4fa3-f199-4a1...,2021-08-08,Ligula Elit Preti...,[antique shops - ...,Tracy Short,1636 Davis Plains,Female,Western Austraia,836.7966870108314,235.5527510544241,1437.028984063036,42.581815939994776,1530.0211433587413,2.4394982812246875,2478.71725636885,28.07392873978152,6548.73302066646,null,null,null,null,null,null,null,null,null,null,null,WA
604031097,7466,165107,11702,53918538787,77134.2146046811,8584d8d1-061b-41c...,2021-07-27,In Tempus Inc.,[antique shops - ...,Ana Robinson,77844 Garcia Vall...,Female,West Coast (Tas.),548.0,160.0,650.0,47.0,958.0,2.0,1030.0,16.0,4267.0,208,462,172,200,521,454,629,726,576,254,59,TAS
604031097,7466,165107,11702,53918538787,77134.2146046811,8584d8d1-061b-41c...,2021-07-27,In Tempus Inc.,[antique shops - ...,Ana Robinson,77844 Garcia Vall...,Female,West Coast (Tas.),548.0,160.0,650.0,47.0,958.0,2.0,1030.0,16.0,4267.0,208,462,172,200,521,454,629,726,576,254,59,TAS
401051019,5069,1277505,10428,54860127682,51618.337208028475,572fa5c4-72f8-4ad...,2021-07-17,Laoreet Posuere E...,[antique shops - ...,Richard Jones,5032 William Glen,Male,St Peters - Marden,954.0,320.0,2080.0,41.0,1879.0,2.299999952316284,8201.0,77.0,13515.0,595,1453,769,856,1942,1682,1725,1851,1393,829,416,SA
401051019,5069,1277505,10428,54860127682,51618.337208028475,572fa5c4-72f8-4ad...,2021-07-17,Laoreet Posuere E...,[antique shops - ...,Richard Jones,5032 William Glen,Male,St Peters - Marden,954.0,320.0,2080.0,41.0,1879.0,2.299999952316284,8201.0,77.0,13515.0,595,1453,769,856,1942,1682,1725,1851,1393,829,416,SA
401051019,5069,1277505,10428,54860127682,51618.337208028475,572fa5c4-72f8-4ad...,2021-07-17,Laoreet Posuere E...,[antique shops - ...,Richard Jones,5032 William Glen,Male,St Peters - Marden,954.0,320.0,2080.0,41.0,1879.0,2.299999952316284,8201.0,77.0,13515.0,595,1453,769,856,1942,1682,1725,1851,1393,829,416,SA
401051019,5069,1277505,10428,54860127682,51618.337208028475,572fa5c4-72f8-4ad...,2021-07-17,Laoreet Posuere E...,[antique shops - ...,Richard Jones,5032 William Glen,Male,St Peters - Marden,954.0,320.0,2080.0,41.0,1879.0,2.299999952316284,8201.0,77.0,13515.0,595,1453,769,856,1942,1682,1725,1851,1393,829,416,SA
401051019,5069,1277505,10428,54860127682,51618.337208028475,572fa5c4-72f8-4ad...,2021-07-17,Laoreet Posuere E...,[antique shops - ...,Richard Jones,5032 William Glen,Male,St Peters - Marden,954.0,320.0,2080.0,41.0,1879.0,2.299999952316284,8201.0,77.0,13515.0,595,1453,769,856,1942,1682,1725,1851,1393,829,416,SA
401051019,5069,1277505,10428,54860127682,51618.337208028475,572fa5c4-72f8-4ad...,2021-07-17,Laoreet Posuere E...,[antique shops - ...,Richard Jones,5032 William Glen,Male,St Peters - Marden,954.0,320.0,2080.0,41.0,1879.0,2.299999952316284,8201.0,77.0,13515.0,595,1453,769,856,1942,1682,1725,1851,1393,829,416,SA


Al instances above $50,000 come from anitque shops or jewelry shops and thus is plausible for the individuals to be spending large amounts of money. 

## Order Date 

In [85]:
print(data.agg({'order_datetime': 'min'}))
print(data.agg({'order_datetime': 'max'}))

+-------------------+
|min(order_datetime)|
+-------------------+
|         2021-02-28|
+-------------------+

+-------------------+
|max(order_datetime)|
+-------------------+
|         2021-08-27|
+-------------------+



Based on the above all values are in 2021.

## Gender

In [86]:
from pyspark.sql.functions import count

data.groupBy('gender').count()

gender,count
Undisclosed,3786857
Female,16928185
Male,17129766


## Median personal income

In [87]:
print(data.agg({'Median_tot_prsnl_inc_weekly': 'min'}))
print(data.agg({'Median_tot_prsnl_inc_weekly': 'max'}))

+--------------------------------+
|min(Median_tot_prsnl_inc_weekly)|
+--------------------------------+
|                             0.0|
+--------------------------------+



+--------------------------------+
|max(Median_tot_prsnl_inc_weekly)|
+--------------------------------+
|                         92500.0|
+--------------------------------+



## Median rent weekly

In [88]:
print(data.agg({'Median_rent_weekly': 'min'}))
print(data.agg({'Median_rent_weekly': 'max'}))

+-----------------------+
|min(Median_rent_weekly)|
+-----------------------+
|                    0.0|
+-----------------------+



+-----------------------+
|max(Median_rent_weekly)|
+-----------------------+
|                 1200.0|
+-----------------------+



## Median mortgage monthly

In [89]:
print(data.agg({'Median_mortgage_repay_monthly': 'min'}))
print(data.agg({'Median_mortgage_repay_monthly': 'max'}))

+-----------------------+
|min(Median_rent_weekly)|
+-----------------------+
|                    0.0|
+-----------------------+



+-----------------------+
|max(Median_rent_weekly)|
+-----------------------+
|                 1200.0|
+-----------------------+



## Median age

In [ ]:
print(data.agg({'Median_age_persons': 'min'}))
print(data.agg({'Median_age_persons': 'max'}))

## Median household income

In [ ]:
print(data.agg({'Median_tot_hhd_inc_weekly': 'min'}))
print(data.agg({'Median_tot_hhd_inc_weekly': 'max'}))

## Average household size

In [ ]:
print(data.agg({'Average_household_size': 'min'}))
print(data.agg({'Average_household_size': 'max'}))

## Average household size

In [ ]:
print(data.agg({'Average_household_size': 'min'}))
print(data.agg({'Average_household_size': 'max'}))

no outliers found

In [90]:
data.write.mode('overwrite').parquet('../data/curated/outlier_data.parquet')